In [2]:
import pandas as pd
df = pd.read_csv('C:/Users/niraj/Downloads/DS_Ml_Course/MachineLearning/USED_car_price_Predication/used_cars_dataset_v2.csv')

In [3]:
df.head()

,Brand,model,Year,Age,kmDriven,Transmission,Owner,FuelType,PostedDate,AdditionInfo,AskPrice
0,Honda,City,2001,23,"98,000 km",Manual,second,Petrol,Nov-24,"Honda City v teck in mint condition, valid gen...","₹ 1,95,000"
1,Toyota,Innova,2009,15,190000.0 km,Manual,second,Diesel,Jul-24,"Toyota Innova 2.5 G (Diesel) 7 Seater, 2009, D...","₹ 3,75,000"
2,Volkswagen,VentoTest,2010,14,"77,246 km",Manual,first,Diesel,Nov-24,"Volkswagen Vento 2010-2013 Diesel Breeze, 2010...","₹ 1,84,999"
3,Maruti Suzuki,Swift,2017,7,"83,500 km",Manual,second,Diesel,Nov-24,Maruti Suzuki Swift 2017 Diesel Good Condition,"₹ 5,65,000"
4,Maruti Suzuki,Baleno,2019,5,"45,000 km",Automatic,first,Petrol,Nov-24,"Maruti Suzuki Baleno Alpha CVT, 2019, Petrol","₹ 6,85,000"


In [10]:

df['kmDriven'] = (
    df['kmDriven'].astype(str)  # Convert all values to string
    .str.replace(' km', '', regex=False)  # Remove ' km'
    .str.replace(',', '', regex=False)    # Remove commas
)

# Convert back to numeric (handling errors for NaNs)
df['kmDriven'] = pd.to_numeric(df['kmDriven'], errors='coerce')

# Optionally fill missing values (NaNs) with median
df['kmDriven'] = df['kmDriven'].fillna(df['kmDriven'].median()).astype(int)
# Map categorical columns
df['Brand'] = df['Brand'].astype('category').cat.codes
df['model'] = df['model'].astype('category').cat.codes

# Map Transmission
df['Transmission'] = df['Transmission'].map({'Manual': 0, 'Automatic': 1})

# Map FuelType
df['FuelType'] = df['FuelType'].map({
    'Petrol': 0, 'Diesel': 1, 'CNG': 2, 'LPG': 3, 'Electric': 4
})

# Map Owner
df['Owner'] = df['Owner'].map({
    'First Owner': 0, 
    'Second Owner': 1, 
    'Third Owner': 2, 
    'Fourth & Above Owner': 3, 
    'Test Drive Car': 4
})

# Fill any remaining NaNs
df = df.fillna(0)

# Features and Target
X = df[['Brand', 'model', 'Year', 'kmDriven', 'Transmission', 'FuelType', 'Owner']]
df['AskPrice'] = (
    df['AskPrice']
    .astype(str)  # Ensure all values are strings
    .str.replace(r'[₹,]', '', regex=True)  # Remove ₹ and commas
    .replace('nan', '0')  # Handle missing values
    .astype(float)  # Convert to float
)

df['AskPrice'] = (
    df['AskPrice'].astype(str)
    .str.replace(r'[₹,]', '', regex=True)
    .replace('nan', '0')
    .astype(float)
)

# Split features and target
X = df[['Brand', 'model', 'Year', 'kmDriven', 'Transmission', 'FuelType', 'Owner']]
y = df['AskPrice']  # make sure this is now numeric

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [17]:
#display Brand column unique values
df['Brand'].unique()

array(['Honda', 'Toyota', 'Volkswagen', 'Maruti Suzuki', 'BMW', 'Ford',
       'Kia', 'Mercedes-Benz', 'Hyundai', 'Audi', 'Renault', 'MG',
       'Volvo', 'Skoda', 'Tata', 'Mahindra', 'Mini', 'Land Rover', 'Jeep',
       'Chevrolet', 'Jaguar', 'Fiat', 'Aston Martin', 'Porsche', 'Nissan',
       'Force', 'Mitsubishi', 'Lexus', 'Isuzu', 'Datsun', 'Ambassador',
       'Rolls-Royce', 'ICML', 'Bajaj', 'Opel', 'Ashok', 'Bentley',
       'Ssangyong', 'Maserati', 'Toyota Land', 'Citroen', 'Lamborghini',
       'Hummer'], dtype=object)

In [23]:
df['Brand'].unique()


array([13, 39, 41, 26,  4, 12, 20, 28, 15,  3, 34, 24, 42, 36, 38, 25, 29,
       22, 19,  7, 18, 10,  2, 33, 31, 11, 30, 23, 17,  9,  0, 35, 16,  5,
       32,  1,  6, 37, 27, 40,  8, 21, 14], dtype=int8)

In [11]:
# To find mean squared error
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
y_pred = model.predict(X_test)  
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")   
print(f"Mean Absolute Error: {mae}")


Mean Squared Error: 1842528197267.069
R^2 Score: 0.08805874705342864
Mean Absolute Error: 709688.6851564224


In [12]:
import pickle
# Save the model to a file
with open('car_price_model.pkl', 'wb') as file:
    pickle.dump(model, file)
# Flask app to use the model for prediction